## Locality Sensitive Hashing
(by Tevfik Aytekin)

### Approximate Nearest Neighbor

Finding nearest neighbors in a set of objects is a very general problem which has applications in many areas. If the size of the set of objects is very large then an exhaustive pairwise comparision of all ojects can be very costly.

**Randomized near-neighbor reporting:** Given a set $P$ of points in a $d$-dimensional space $\mathbb{R}^d$, and parameters $R > 0, > δ > 0$, construct a data structure which, given any query point $q$, reports each $R$-near neighbor of $q$ in $P$ with probability $1 − δ$.


### The main idea of LSH

The main idea of LSH is to design hash functions such that the probability of a collision is much higher for closer points compared to points which are far apart. Given such hash functions one can hash a query point and retrive the elements in the buckets that contain the query point.


### Formal Definition of LSH 
(following definition and figure is taken from [mmds book](http://www.mmds.org/) where you can read more about LSH)

Let $d_1 < d_2$ be two distances according to some distance measure $d$. A family $F$ of functions is said to be $(d_1, d_2, p_1, p_2)-sensitive$ if for every $f$ in $F$:
1. If $d(x,y) ≤ d_1$, then the probability that $f(x) = f(y)$ is at least $p_1$. 
2. If $d(x,y) ≥ d_2$, then the probability that $f(x) = f(y)$ is at most $p_2$.

In order for a LSH family to be useful: $p_1 > p_2$.

<img src="images/lsh.png" width = "400">

Once you have a family $F$ of functions we can amplify the gap between the $p_1$ and $p_2$ by AND and OR constructions. 

Suppose we are given a $(d_1, d_2, p_1, p_2)-sensitive$ family $F$. We can make an AND construction by building $g=(f_1,f_2,...,f_k)$ by choosing $k$ functions at random from $F$. We say that $g(x) = g(y)$ iff $g_i(x) = g_i(y)$ for all $i = 1, 2, . . . , k$.

We can make OR constructions by building $h=(g_1,f_2,...,g_l)$ by choosing $l$ functions from the set of $g$ functions as defined above. at random from $F$. Note that all the $f_i$'s in each $g_j$ are choosen at random from $F$. We define $h(x) = h(y)$ iff $g_i(x) = g_i(y)$ for one or more values of $i$. 

We assume a seperate hash table for each $g_i$. We will insert every point $p$ by the key $q_i(p)$ into each bucket of $g_i$'s during the build phase of LSH. When a query point $q$ comes we will return the elemements in the buckets at each $g_i(q)$


### An LSH Function for Cosine Similarity

For every distance metric (such as cosine, Jaccard, hamming, etc.) you should define a LSH family of hash functions. For cosine distance sign of the dot product of the data point with a random unit vector is used for construction the LSH. Each such random unit vector constitutes a different function in the LSH family F. The following figure illustrates why dot product with a random unit vector can be used to build an LSH family of function for cosine distance. 

<img src="images/lsh_cosine.jpg" width = "400">

In [13]:
import numpy as np
from scipy.spatial import distance
import time
from sklearn.metrics.pairwise import pairwise_distances
import itertools
import pandas as pd
#from tabulate import tabulate





### Performance measurement

Nearest neighbor search: Suppose that we have a large dataset of vectors X and given a target vector we want to find the most similar k vectors to the target vector in the dataset X. Note that we can do this type of search more than once. 

First let us generate the dataset X:

In [14]:
n_vectors = 100000
dim = 10
n_neighbors = 100
n_queries = 2000

target_vectors = np.random.randn(n_queries, dim)
dataset = np.random.randn(n_vectors, dim)

In [25]:
ns = NaiveSearch(dataset)
ns.build()
tic = time.time()
ns_nn = []
for i in range(n_queries):
    target_vector = target_vectors[i,:].reshape(1,dim)
    neighbors = ns.find_nn(target_vector, n_neighbors)
    ns_nn.append(neighbors)
toc = time.time()
print("Time:"+str(1000*(toc-tic))+"ms")

Time:20566.476106643677ms


In [32]:

#target_vector_b = target_vector
#print(neighbors)
min_n_random_vectors = 10
max_n_random_vectors = 15
max_n_bands = 5

elapsed_time = np.zeros((max_n_bands, (max_n_random_vectors-min_n_random_vectors)+1))
recall = np.zeros((max_n_bands, (max_n_random_vectors-min_n_random_vectors)+1))
avg_neigbor_size = np.zeros((max_n_bands, (max_n_random_vectors-min_n_random_vectors)+1))

for b in range(1, max_n_bands+1):
    for v in range(min_n_random_vectors,max_n_random_vectors+1):
        lsh_nn = []
        lsh = LSH(dataset)
        lsh.build(v, n_bands = b)
        tic = time.process_time()
        for i in range(n_queries):
            target_vector = target_vectors[i,:].reshape(1,dim)
            neighbors = lsh.find_nn(target_vector, n_neighbors, n_bands = b)
            lsh_nn.append(neighbors)
        toc = time.process_time()
        print(b-1,v-min_n_random_vectors)
        print(v)
        elapsed_time[b-1,v-min_n_random_vectors] = (toc-tic)*1000
        true_positives = sum([len(np.intersect1d(ns_nn[i], lsh_nn[i][:n_neighbors])) for i in range(n_queries)])
        recall[b-1,v-min_n_random_vectors] = true_positives / (n_queries*n_neighbors)
        avg_neigbor_size[b-1,v-min_n_random_vectors] = len(list(itertools.chain(*lsh_nn)))/n_queries
    


#pd.DataFrame({"Number of Random Vectors:": range(min_n_random_vectors,max_n_random_vectors+1), "Elapsed Time":elapsed_time, "Recall": recall, "Avg. Neighbor Size": avg_neigbor_size}) 
elapsed_time_df = pd.DataFrame(elapsed_time)
recall_df = pd.DataFrame(recall)
avg_neigbor_size_df = pd.DataFrame(avg_neigbor_size)

row_names = [str(i) for i in range(1, max_n_bands+1)]
column_names = [str(i) for i in range(min_n_random_vectors, max_n_random_vectors+1)]
elapsed_time_df.columns = recall_df.columns = avg_neigbor_size_df.columns = column_names
elapsed_time_df.index = recall_df.index = avg_neigbor_size_df.index = row_names
print("Elapsed Time")
display(elapsed_time_df)
print("Recall")
display(recall_df)
print("Avg. Neighbor Size")
display(avg_neigbor_size_df)

0 0
10
0 1
11
0 2
12
0 3
13
0 4
14
0 5
15
1 0
10
1 1
11
1 2
12
1 3
13
1 4
14
1 5
15
2 0
10
2 1
11
2 2
12
2 3
13
2 4
14
2 5
15
3 0
10
3 1
11
3 2
12
3 3
13
3 4
14
3 5
15
4 0
10
4 1
11
4 2
12
4 3
13
4 4
14
4 5
15
Elapsed Time


,10,11,12,13,14,15
1,2251.501,2281.903,1881.211,716.468,715.501,640.685
2,4246.054,3099.028,2631.128,1587.320,1901.274,722.371
3,4561.662,3361.857,2754.723,2298.846,1595.129,1142.491
4,5469.603,3684.521,2880.174,2581.373,2066.969,1722.599
5,5863.859,4703.014,3436.484,3047.911,2486.806,2136.813


Recall


,10,11,12,13,14,15
1,0.149735,0.137700,0.109220,0.084290,0.076830,0.060785
2,0.297170,0.241765,0.203355,0.159500,0.149875,0.110730
3,0.385170,0.329770,0.277095,0.245390,0.199555,0.169950
4,0.472820,0.395250,0.341090,0.308390,0.256225,0.211770
5,0.532090,0.476835,0.423845,0.361585,0.304265,0.272705


Avg. Neighbor Size


,10,11,12,13,14,15
1,376.1320,397.0000,232.8455,118.2515,107.4095,60.6275
2,1167.7910,663.4810,444.0195,207.8125,216.2850,97.4130
3,1351.5915,812.6195,546.9365,436.7610,245.7855,187.5365
4,1791.2980,955.5080,681.5155,566.5705,337.8880,222.3765
5,1912.0350,1361.0680,996.6785,660.3095,402.4205,367.8485


In [47]:
type(elapsed_time_df)

pandas.core.frame.DataFrame

In [42]:
[str(i) for i in range(1, max_n_bands+1)]

['1', '2']

In [17]:
class NaiveSearch:
    def __init__(self, data):
        # data is a n-by-d matrix where d is the length of the vectors
        # and n is the number of vectors. 
        self.data = data
        self.norms = None
        self.data_normalized_T = None
    
        
    def build(self):
        self.norms = np.linalg.norm(self.data, axis=1)
        self.norms.shape = (len(self.norms), 1)
        
        data_normalized = np.divide(self.data, self.norms)
        self.data_normalized_T = data_normalized.T
            
            
    def find_nn(self, target_vector, n_neighbors=10):

        #target_vector_normalized = np.linalg.norm(target_vector)
        
        sims = np.dot(target_vector,self.data_normalized_T)[0]
        return sims.argsort()[::-1][:n_neighbors]


In [27]:

class LSH:
    def __init__(self, data):
        # data is a n-by-d matrix where d is the length of the vectors
        # and n is the number of vectors. 
        self.data = data
        self.bands = []
        self.random_vectors = []
    
    def build(self, n_random_vectors, n_bands = 1):
        for b in range(n_bands):
            # generate random vectors
            self.bands.append({})
            dim = self.data.shape[1]
            self.random_vectors.append(np.random.randn(n_random_vectors, dim))
            # generate dim-by-n index bits
            sign_bits = np.dot(self.data, self.random_vectors[b].T) >= 0
            n_data_vectors = self.data.shape[0]
            for i in range(n_data_vectors):
                key = tuple(sign_bits[i,:])
                if key not in self.bands[b]:
                    self.bands[b][key] = []
                self.bands[b][key].append(i)
            
            
    def find_nn(self, target_vector, n_neighbors=10, n_bands = 1):

        candidate_ids = []
        for b in range(n_bands):
            sign_bits = (np.dot(target_vector, self.random_vectors[b].T) >= 0).flatten()
            sign_bits_tuple = tuple(sign_bits)
            ids = self.bands[b].get(sign_bits_tuple)
            if ids is None: 
                ids = []
            candidate_ids = candidate_ids + ids
        if len(candidate_ids) > 0:
                candidate_vectors = self.data[candidate_ids, :]
                sims = 1 - pairwise_distances(target_vector, candidate_vectors, metric='cosine').flatten()
                sorted_nn = sims.argsort()[::-1]
                return np.array([candidate_ids[i] for i in sorted_nn])
        else:
            return []
            
        

### Making sense of the cost of pairwise similarity computation

Suppose that we have n objects represented as vectors of size d. The cost of computing all pairwise similarities is $O(n^2d)$. When $n$ is large the cost of this computation can be quite large. And in some applications like finding near duplicate web pages, in order to eliminate them from search results, $n$ (number of web pages) can be really large.

To get a sense of this cost below is a simple code which measures the time two multiply two matrices of size $n$-by-$d$. (Note that some version of multiplication is needed in order to find similarities between the vectors)

In [24]:
n = 10000
d = 1000
X = np.random.randn(n,d)
tic = time.process_time()

z = np.dot(X,X.T)

toc = time.process_time()
print("Time:"+str(1000*(toc-tic))+"ms")

Time:4142.167ms


Since the time complexity is quadratic, we expect the figures in the following table if we increase $n$. You can try some larger $n$'s to test this.

|  n | time  | 
|:---|:---|
|  100k | 400 seconds  |
|  1m | 11 hours  |
|  10m | 46 days  |
|  100m | 12 years  |
|  1b | 12 centuries |

The above results are taken on a 2,3 GHz Dual-Core Intel Core i5 laptop.